## Agenda
- 리인덱싱
- 삭제
- 연산
- 정렬

In [1]:
import pandas as pd
import numpy as np

## 1.2 핵심 기능

### 1.2.1 리인덱싱

#### 시리즈 리인덱싱

In [2]:
obj = pd.Series([4.5, 7.2, -5.3, 3.6], index=['d', 'b', 'a', 'c'])
obj

d    4.5
b    7.2
a   -5.3
c    3.6
dtype: float64

- 새로운 인덱스가 추가되면 `NaN`이 사용

In [4]:
obj.reindex(['a','b','c','d','e'])

a   -5.3
b    7.2
c    3.6
d    4.5
e    NaN
dtype: float64

- 지정되지 않은 인덱스는 무시

In [5]:
obj.reindex(['a','c','d','e'])

a   -5.3
c    3.6
d    4.5
e    NaN
dtype: float64

#### 결측치 채우기 1: `method` 키워드 인자

- 리인덱싱 과정에서 결측치가 발생할 때 여러 방식으로 채울 수 있다.
- `method='fill'` 키워드 인자는 결측치를 위쪽에 위치한 값으로 채운다.

__주의사항:__ 인덱스가 오름 또는 내림 차순으로 정렬되어 있는 경우에만 가능하다.

In [6]:
obj3 = pd.Series(['blue', 'purple', 'yellow'], index=[0, 2, 5])
obj3

0      blue
2    purple
5    yellow
dtype: object

In [7]:
obj3.reindex(range(6))

0      blue
1       NaN
2    purple
3       NaN
4       NaN
5    yellow
dtype: object

In [8]:
obj3.reindex(range(6),method="ffill") # ffill 결측치 위쪽에 위치한 값으로 채움

0      blue
1      blue
2    purple
3    purple
4    purple
5    yellow
dtype: object

- 아랫쪽에 있는 값으로 채울 수도 있다.
- `method='bfill'`

In [9]:
obj3.reindex(range(6),method="bfill")

0      blue
1    purple
2    purple
3    yellow
4    yellow
5    yellow
dtype: object

- 아니면 가장 가까운 곳에 있는 값으로 채울 수도 있다.
- `method='nearest'`

In [13]:
obj3.reindex(range(-2, 6), method ="nearest")

-2      blue
-1      blue
0       blue
1     purple
2     purple
3     purple
4     yellow
5     yellow
dtype: object

In [ ]:
# 예를 들어 타이타닉 사망자 조사를 하는 프로젝트를 진행한다고 가정할 때,
#  주변값의 예측치를 통해 그 값을 채워넣을 수 있다.

#### 결측치 채우기 2: `fill_value` 키워드 인자

- 리인덱싱 과정에서 발생하는 모든 결측치를 지정된 값으로 대체
- 기본값은 `NaN` 이다.

In [14]:
obj3.reindex(range(-1, 6), fill_value="No Color")

-1    No Color
0         blue
1     No Color
2       purple
3     No Color
4     No Color
5       yellow
dtype: object

- 리인덱싱은 항상 새로운 시리즈를 생성한다.
- 따라서 `obj3` 자체는 변하지 않는다.

#### 데이터프레임 리인덱싱

- 데이터프레임은 `index`와 `columns` 속성에 대해 리인덱싱이 가능하며 작동법은 Series의 인덱싱과 동일하다.

In [15]:
frame = pd.DataFrame(np.arange(9).reshape((3, 3)),
                     index=['a', 'c', 'd'],
                     columns=['Ohio', 'Texas', 'California'])
frame

,Ohio,Texas,California
a,0,1,2
c,3,4,5
d,6,7,8


- `reindex()` 메서드는 기본적으로 행의 `index` 에 대해 작동

In [16]:
frame2 = frame.reindex(["a","b","c","d"])

In [17]:
frame2

,Ohio,Texas,California
a,0.0,1.0,2.0
b,NaN,NaN,NaN
c,3.0,4.0,5.0
d,6.0,7.0,8.0


- 열의 `columns`에 대해서는 `columns` 키워드 인자를 활용

In [18]:
states = ['Texas', 'Utah', 'California']
frame.reindex(columns=states)

,Texas,Utah,California
a,1,NaN,2
c,4,NaN,5
d,7,NaN,8


### 5.2.2 `drop()` 메서드

- 특정 행 또는 열의 인덱스를 제외한 나머지로 이루어진 시리즈/데이터프레임을 구할 때 사용
- 시리즈의 경우 인덱스를 한 개 또는 여러 개 지정하면 나머지로 이루어진 시리즈를 얻을수 있다. 

In [19]:
pd_1 = pd.Series(np.arange(5), index = ['a','b','c','d','e'])
print(pd_1)

a    0
b    1
c    2
d    3
e    4
dtype: int64


In [21]:
pd_1.drop("c")

a    0
b    1
d    3
e    4
dtype: int64

In [25]:
pd_1.drop(["d","c"])

a    0
b    1
e    4
dtype: int64

- `inplace=True` 키워드 인자를 이용하여 원본 수정 가능
- 사용시 주의 

In [27]:
pd_1.drop("c", inplace=True)

- 데이터프레임의 경우도 기본적으로 행의 인덱스를 기준으로 작동한다.

In [26]:
data = pd.DataFrame(np.arange(16).reshape((4, 4)),
                    index=['Ohio', 'Colorado', 'Utah', 'New York'],
                    columns=['one', 'two', 'three', 'four'])
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


In [28]:
data.drop(["Colorado","Ohio"])

,one,two,three,four
Utah,8,9,10,11
New York,12,13,14,15


- 열을 기준으로 작동하게 하려면 `axis=1`로 지정한다.

In [31]:
data.drop(["two"],axis =1)

,one,three,four
Ohio,0,2,3
Colorado,4,6,7
Utah,8,10,11
New York,12,14,15


- `axis='columns'` 지정

In [32]:
data.drop(["two"],axis ="columns")

,one,three,four
Ohio,0,2,3
Colorado,4,6,7
Utah,8,10,11
New York,12,14,15


### 1.2.3 인덱싱, 슬라이싱, 필터링(부울 인덱싱)

#### 시리즈의 인덱싱, 슬라이싱, 필터링(부울 인덱싱)

시리즈의 경우 1차원 넘파이 어레이와 거의 동일하게 작동한다.
다만 정수 대신에 지정된 인덱스를 사용할 때 조금 차이가 있다.

In [33]:
obj = pd.Series(np.arange(4.), index=['a', 'b', 'c', 'd'])
obj

a    0.0
b    1.0
c    2.0
d    3.0
dtype: float64

In [34]:
obj["b"]

1.0

In [35]:
obj[1]

1.0

- 여러 개의 인덱스를 리스트로 지정하여 인덱싱을 진행

In [36]:
obj[["a","b"]]

a    0.0
b    1.0
dtype: float64

In [37]:
obj[["c","a","b"]]

c    2.0
a    0.0
b    1.0
dtype: float64

- 필터링(부울 인덱싱)은 동일하게 작동

In [39]:
type(obj<2)

pandas.core.series.Series

In [40]:
obj[obj<2]

a    0.0
b    1.0
dtype: float64

- Label 슬라이싱

In [41]:
obj[["b","c"]]

b    1.0
c    2.0
dtype: float64

In [42]:
obj["b":"c"] # 문자 슬라이싱은 양끝 구간을 모두 포함한다.

b    1.0
c    2.0
dtype: float64

In [45]:
#obj_2 = obj.reindex(["b","d","c","a"])

In [47]:
#obj_2

b    1.0
d    3.0
c    2.0
a    0.0
dtype: float64

In [46]:
#obj_2["b":"d"]

b    1.0
d    3.0
dtype: float64

#### 데이터프레임의 인덱싱, 슬라이싱, 필터링(부울 인덱싱)

In [48]:
data = pd.DataFrame(np.arange(16).reshape((4, 4)),
                    index=['Ohio', 'Colorado', 'Utah', 'New York'],
                    columns=['one', 'two', 'three', 'four'])
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


In [49]:
data["two"]

Ohio         1
Colorado     5
Utah         9
New York    13
Name: two, dtype: int64

In [51]:
data[["two","three"]]

,two,three
Ohio,1,2
Colorado,5,6
Utah,9,10
New York,13,14


In [52]:
# 숫자 슬라이싱은 행을 기준으로 동작한다.
data[:2]

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7


In [53]:
mask = data["three"]>5

In [54]:
data[mask]

,one,two,three,four
Colorado,4,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


In [55]:
# 필터링을 이용해 특정 값을 업데이트 할 수 있다.
data[~mask] = 3 # mask 조건을 충족하지 않는 값들에 3을 대입

In [56]:
data

,one,two,three,four
Ohio,3,3,3,3
Colorado,4,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


#### 행 단위 인덱싱/슬라이싱

`loc()` 또는 `iloc()` 메서드를 이용한다.

- `loc()` 메서드: 라벨을 이용할 경우
- `iloc()` 메서드: 정수 인덱스를 이용할 경우

In [57]:
data.loc["Colorado"]

one      4
two      5
three    6
four     7
Name: Colorado, dtype: int64

In [58]:
data.iloc[2]

one       8
two       9
three    10
four     11
Name: Utah, dtype: int64

In [59]:
data[1:2]

,one,two,three,four
Colorado,4,5,6,7


In [ ]:
# 행과 열을 동시에 인덱싱, 슬라이싱 하기 위해서는 loc 또는 iloc 메소드를 사용한다.

In [60]:
data.loc["Colorado",["two","three"]]

two      5
three    6
Name: Colorado, dtype: int64

In [61]:
data.iloc[2,[0,1,2]]

one       8
two       9
three    10
Name: Utah, dtype: int64

In [ ]:
# data.iloc[2, [0:2]] 오류가 난다. iloc로 인덱싱과 슬라이싱을 동시에 사용할 떄 괄호를 벗겨준다.
data.iloc[2, 0:2]

### 5.2.5 산술 연산

시리즈/데이터프레임의 사칙 연산은 기본적으로 아래 원칙을 따른다.

- 연산에 사용된 모든 인덱스는 포함
- 공통으로 사용되는 인덱스의 항목에 대해서만 연산 적용. 그렇지 않으면 `NaN`으로 처리.

In [62]:
s1 = pd.Series([7.3, -2.5, 3.4, 1.5], index=['a', 'c', 'd', 'e'])

In [64]:
s2 = pd.Series([-2.1, 3.6, -1.5, 4, 3.1],
               index=['a', 'c', 'e', 'f', 'g'])

In [65]:
s1+s2

a    5.2
c    1.1
d    NaN
e    0.0
f    NaN
g    NaN
dtype: float64

In [66]:
df_1 = pd.DataFrame(np.arange(9).reshape((3, 3)),
                    index=['Ohio', 'Colorado', 'New York'],
                    columns=['two', 'three', 'four'])
df_2 = pd.DataFrame(np.arange(16).reshape((4, 4)),
                    index=['Ohio', 'Colorado', 'Utah', 'New York'],
                    columns=['one', 'two', 'three', 'four'])

In [67]:
df_1+df_2

,four,one,three,two
Colorado,12.0,NaN,10.0,8.0
New York,23.0,NaN,21.0,19.0
Ohio,5.0,NaN,3.0,1.0
Utah,NaN,NaN,NaN,NaN


- 기본적으로 사용되는 산술 연산 기호에 해당하는 메서드가 존재한다.

| 메서드 | 설명 |
| :--- | :--- |
| `add()` | 덧셈(`+`) 계산 메서드 | 
| `sub()` | 뺄셈(`-`) 계산 메서드 | 
| `mul()` | 곱셈(`*`) 계산 메서드 | 
| `div()` | 나눗셈(`/`) 계산 메서드 | 
| `floordiv()` | 몫 (`//`) 계산 메서드 | 
| `pow()` | 거듭제곱(`**`) 메서드 | 

#### 연산 과정에서 결측치 채우기
- 공통 인덱스가 아니거나 결측치가 이미 존재하는 경우 기본적으로 결측치로 처리됨.
- `fill_value` 키워드 인자를 이용하여 지정된 값으로 처리하게 만들 수도 있다.

In [71]:
df_1.add(df_2, fill_value = 10)

,four,one,three,two
Colorado,12.0,14.0,10.0,8.0
New York,23.0,22.0,21.0,19.0
Ohio,5.0,10.0,3.0,1.0
Utah,21.0,18.0,20.0,19.0


#### 데이터프레임과 시리즈 사이의 연산

넘파이에서 2차원 어레이와 1차원 어레이 사이에
브로드캐스팅이 가능한 경우,
즉, 차원을 맞출 수 있는 경우에 연산이 가능했다.

In [72]:
arr_1 = np.arange(12).reshape((3,4))
print(arr_1)

[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]]


In [73]:
arr_1[0]

array([0, 1, 2, 3])

In [74]:
arr_1-arr_1[0]

array([[0, 0, 0, 0],
       [4, 4, 4, 4],
       [8, 8, 8, 8]])

In [75]:
arr_1[:, 1]

array([1, 5, 9])

In [76]:
arr_1 + arr_1[0]

array([[ 0,  2,  4,  6],
       [ 4,  6,  8, 10],
       [ 8, 10, 12, 14]])

In [78]:
# 굳이 다른 길이가 다른 행을 끌고 와서 연산을 하고 싶을 경우에는 newaxis라는 툴을 사용할 수 있다.
arr_1[:,1][:,np.newaxis].shape

(3, 1)

In [80]:
arr_1[:,1].reshape(3,1)

array([[1],
       [5],
       [9]])

In [79]:
# 근데 reshape를 사용하여 조절할 수 있기 때문에 굳이 그럴 필요가 있을까 한다.
np.expand_dims(arr_1[:,1], axis=-1)

array([[1],
       [5],
       [9]])

In [ ]:
# 브로드 캐스팅 / 공통 인덱스가 존재하면 dataframe과 series 사이의 브로드 캐스팅이 적용되어 연산된다.


### 1.2.6 함수 적용

#### 유니버설 함수

유니버설 함수는 넘파이의 경우와 동일하게 작동한다.

In [81]:
frame = pd.DataFrame(np.random.randn(4, 3), columns=list('bde'),
                     index=['Utah', 'Ohio', 'Texas', 'Oregon'])
frame

,b,d,e
Utah,0.582200,1.131728,-1.420766
Ohio,-0.585524,0.260996,-0.049774
Texas,-0.021007,0.745176,0.418855
Oregon,-0.424232,0.752978,-0.061718


- 넘파이의 `abs()` 함수를 적용하면 항목별로 이루어진다.

In [82]:
np.abs(frame)

,b,d,e
Utah,0.582200,1.131728,1.420766
Ohio,0.585524,0.260996,0.049774
Texas,0.021007,0.745176,0.418855
Oregon,0.424232,0.752978,0.061718


- 시리즈에 대해서도 동일하게 동작

In [84]:
np.abs(frame["b"])

Utah      0.582200
Ohio      0.585524
Texas     0.021007
Oregon    0.424232
Name: b, dtype: float64

#### `map()`과 `applymap()` 메서드 

- 유니버설 함수가 아닌 함수를 `Series`의 항목별 적용 하기위해선 `map()`를 사용한다

__예__ 
- 람다(lambda) 함수를 이용해 부동소수점을 소수점 이하 셋째 자리에서 반올림한 값만 보여주도록 한다.

In [88]:
func_1 = lambda x: float("%.2f" % x)

In [87]:
func_1(3.1456)

'3.15'

In [89]:
frame["b"]

Utah      0.582200
Ohio     -0.585524
Texas    -0.021007
Oregon   -0.424232
Name: b, dtype: float64

- 시리즈에 적용

In [ ]:
frame["b"].map(func_1) # map 은 각 요소별로 함수를 수행하도록 만들어준다.

- 유니버설 함수가 아닌 함수를 `데이터프레임`의 항목별로 적용하려면 `applymap()`를 사용한다.

In [90]:
frame.applymap(func_1)

,b,d,e
Utah,0.58,1.13,-1.42
Ohio,-0.59,0.26,-0.05
Texas,-0.02,0.75,0.42
Oregon,-0.42,0.75,-0.06


#### `apply()` 메서드

- DataFrame의 행 또는 열 단위로 함수를 적용하려면 `apply()` 메서드를 활용한다.
- 기본은 열 단위로 함수가 적용되며 반환값이 스칼라 값이면 시리즈가 반환

__예__ 
- 최댓값과 최소값의 차이를 반환하는 함수를 적용해 보자

In [91]:
# 1000 원 -> function -> 1000 덧셈 뺄셈 bool
def func_2(x):
    tmp = x.max() -x.min()
    return tmp

In [92]:
frame.apply(func_2)

b    1.167724
d    0.870733
e    1.839621
dtype: float64

- 행 별로 함수 적용
    - `axis=1` 또는 `axis='columls'` 지정

In [93]:
frame.apply(func_2,axis =1)

Utah      2.552494
Ohio      0.846520
Texas     0.766183
Oregon    1.177210
dtype: float64

In [ ]:
# apply 적용시에 함수의 반환값이 Series면 apply의 결과는 Dataframe을 반환

In [94]:
def func_3(x):
    tmp = x.max() -x.min()
    return pd.Series([x.min(),x.max(),tmp], index = ["min","max","max-min"])

In [95]:
frame.apply(func_3, axis = 1)

,min,max,max-min
Utah,-1.420766,1.131728,2.552494
Ohio,-0.585524,0.260996,0.846520
Texas,-0.021007,0.745176,0.766183
Oregon,-0.424232,0.752978,1.177210


### 1.2.7 정렬

- 행과 열의 인덱스 또는 항목을 대상으로 정렬할 수 있다.

#### `sort_index()` 메서드
- 시리즈의 경우 인덱스를 기준으로 정렬

In [96]:
frame.sort_index()

,b,d,e
Ohio,-0.585524,0.260996,-0.049774
Oregon,-0.424232,0.752978,-0.061718
Texas,-0.021007,0.745176,0.418855
Utah,0.582200,1.131728,-1.420766


내림차순으로 정렬 : `ascending=False` 키워드 인자 사용

데이터프레임의 경우 행 또는 열의 인덱스를 기준으로 정렬
- 기본은 행의 인데스를 기준으로 정렬
- 열의 인덱스를 기준으로 정렬하려면 `axis=1` 또는 `axis='columns'` 키워드 인자를 사용

In [97]:
df_4 = pd.DataFrame(np.arange(8).reshape((2,4)),
                    index=['three','one'],
                    columns = list("dabc"))

In [99]:
df_4.sort_index(axis=0)

,d,a,b,c
one,4,5,6,7
three,0,1,2,3


In [100]:
df_4.sort_index(axis=0).sort_index(axis=1)

,a,b,c,d
one,5,6,7,4
three,1,2,3,0


- 내림차순으로 정렬하려면 `ascending=False` 키워드 인자를 함께 사용한다.

In [101]:
df_4.sort_index(axis=0,ascending=False).sort_index(axis=1)

,a,b,c,d
three,1,2,3,0
one,5,6,7,4


#### `sort_values()` 메서드
- 지정된 열 또는 행에 속한 값들을 기준으로 정렬할 때 사용
- 데이터프레임의 경우 `by` 키워드 인자를 이용하여 열의 label을 지정

In [103]:
ser_3 = pd.Series([4,56,-7,2])

In [104]:
ser_3.sort_values()

2    -7
3     2
0     4
1    56
dtype: int64

In [106]:
ser_4 = pd.Series([4,56,-7,2,np.nan])
ser_4.sort_values()

2    -7.0
3     2.0
0     4.0
1    56.0
4     NaN
dtype: float64

In [109]:
df_5 = pd.DataFrame({"c":[2,4,1,4,-5],"a":[8,6,3,1,5]})
print(df_5)

   c  a
0  2  8
1  4  6
2  1  3
3  4  1
4 -5  5


In [110]:
df_5.sort_values(by="a")

,c,a
3,4,1
2,1,3
4,-5,5
1,4,6
0,2,8


In [111]:
df_5.sort_values(by=["a","c"])

,c,a
3,4,1
2,1,3
4,-5,5
1,4,6
0,2,8


__참고__ 
- `axis=1`을 이용하여 특정 행의 값을 기준으로 정렬 가능

In [113]:
# sort_values 는 행의 기준으로 정렬하기 위해서는 axis=1로 지정해주면 된다. 딱히 의미가 있지는 않다.
df_5.sort_values(by=3,axis=1)

,a,c
0,8,2
1,6,4
2,3,1
3,1,4
4,5,-5


### 1.2.8 Groupby
- 데이터를 특정 기준으로 그룹핑할 때 활용
- groupby()를 사용할 때는 반드시 aggregate 하는 통계함수와 일반적으로 같이 적용
<img src="https://www.w3resource.com/w3r_images/pandas-groupby-split-apply-combine.svg">

In [114]:
df = pd.DataFrame({
    'city': ['busan', 'busan', 'busan', 'busan', 'seoul', 'seoul', 'seoul'],
    'fruits': ['apple', 'orange', 'banana', 'banana', 'apple', 'apple', 'banana'],
    'price': [100, 200, 250, 300, 150, 200, 400],
    'quantity': [1, 2, 3, 4, 5, 6, 7]
})

In [115]:
df

,city,fruits,price,quantity
0,busan,apple,100,1
1,busan,orange,200,2
2,busan,banana,250,3
3,busan,banana,300,4
4,seoul,apple,150,5
5,seoul,apple,200,6
6,seoul,banana,400,7


- city를 기준으로 price의 평균과 quantity의 평균

In [116]:
df.groupby("city").sum()

,price,quantity
city,,
busan,850,10
seoul,750,18


In [117]:
df.groupby("city").mean()

,price,quantity
city,,
busan,212.5,2.5
seoul,250.0,6.0


- `arrtrgate()`를 이용해 여러 통계함수를 동시에 사용가능

In [119]:
#groupby로 연산하기 위해서는 series 연산이 가능한 함수여야 한다.
df.groupby("city").aggregate([np.mean,np.sum]) 

price      quantity    
        mean  sum     mean sum
city                          
busan  212.5  850      2.5  10
seoul  250.0  750      6.0  18

- 2개 이상의 컬럼으로 그룹핑

In [120]:
df.groupby(["city","fruits"]).mean()

price  quantity
city  fruits                 
busan apple   100.0       1.0
      banana  275.0       3.5
      orange  200.0       2.0
seoul apple   175.0       5.5
      banana  400.0       7.0